## Data preprocessing

## 모델 생성

### fcn(kimm&vib) model

### fcn(curr) model

## Test -kimm/vib/curr 나누기

In [1]:
import numpy as np
import pandas as pd

In [2]:
submission = pd.read_csv('dataSet_Submission_Re/Submission_Commit.csv')

In [3]:
submission = submission.drop(index= 100)

In [4]:
submission_file = submission.reset_index()

In [5]:
submission = submission_file.drop(['index'], axis = 'columns')

In [6]:
Kimm_index = []
Vib_index = []
Curr_index = []
for i in range(len(submission)):
    if submission['Category'][i] == 'Kimm':
        Kimm_index.append(i)
    elif submission['Category'][i] == 'Vibration':
        Vib_index.append(i)
    elif submission['Category'][i] == 'Current':
        Curr_index.append(i)

In [7]:
import os

In [8]:
folder_name = 'dataSet_Submission_Re'
file_list = os.listdir(folder_name)
file_list = file_list[:-2]
print(len(file_list))

151


### kimm

In [10]:
for i, index in enumerate(Kimm_index):
    print(file_list[index])
    one_signal = pd.read_csv(os.path.join(folder_name, file_list[index]))
    one_signal = pd.DataFrame(one_signal['value'].to_numpy().reshape(1,-1))
    if i == 0:
        kimm_df = pd.DataFrame(one_signal)
    else:
        kimm_df = pd.concat([kimm_df, one_signal])


001.csv
002.csv
007.csv
009.csv
011.csv
016.csv
018.csv
020.csv
023.csv
024.csv
026.csv
029.csv
041.csv
043.csv
044.csv
049.csv
053.csv
054.csv
057.csv
062.csv
069.csv
072.csv
074.csv
077.csv
079.csv
084.csv
085.csv
086.csv
099.csv
100.csv
106.csv
109.csv
120.csv
122.csv
123.csv
125.csv
128.csv
130.csv
132.csv
133.csv
134.csv
141.csv
144.csv
149.csv


In [11]:
df_kimm = kimm_df.reset_index()
df_kimm = df_kimm.drop(['index'], axis = 'columns')

In [12]:
df_kimm.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.072470,0.148405,-0.005984,0.080193,-0.012642,0.109416,0.034238,0.012545,0.079226,-0.159403,...,0.058905,0.001762,0.077766,0.025617,0.052918,0.026266,0.076022,0.050075,0.045640,0.032859
1,-0.128280,0.125385,-0.028540,-0.027849,-0.010175,-0.029405,0.086704,0.008864,0.143977,-0.034374,...,0.024438,0.040598,0.017703,0.051307,0.023619,0.041275,0.036324,0.054426,0.046997,0.052025
2,-0.000147,-0.000178,-0.000166,-0.000146,-0.000137,-0.000167,-0.000153,-0.000140,-0.000145,-0.000122,...,-0.000216,-0.000175,-0.000178,-0.000183,-0.000186,-0.000173,-0.000208,-0.000193,-0.000254,-0.000220
3,0.003663,0.015076,0.006914,0.001995,-0.004502,0.010738,0.013627,-0.006417,0.001377,0.001470,...,-0.023124,-0.004702,0.000061,-0.003876,-0.005910,-0.005753,-0.005322,-0.007519,-0.007264,-0.000643
4,0.000007,0.000020,0.000044,0.000047,0.000031,0.000042,0.000051,0.000039,0.000039,0.000054,...,0.000043,0.000034,0.000039,0.000026,0.000051,0.000062,0.000004,0.000052,0.000031,0.000030


In [13]:
print(df_kimm.shape)

(44, 60)


In [14]:
df_kimm.to_csv('kimm_test.csv')

### vibration

In [15]:
for i, index in enumerate(Vib_index):
    print(file_list[index])
    one_signal = pd.read_csv(os.path.join(folder_name, file_list[index]))
    one_signal = pd.DataFrame(one_signal['value'].to_numpy().reshape(1,-1))
    if i == 0:
        vib_df = pd.DataFrame(one_signal)
    else:
        vib_df = pd.concat([vib_df, one_signal])


003.csv
005.csv
008.csv
010.csv
012.csv
014.csv
015.csv
019.csv
021.csv
025.csv
027.csv
028.csv
030.csv
032.csv
033.csv
036.csv
038.csv
039.csv
045.csv
046.csv
048.csv
051.csv
052.csv
056.csv
058.csv
059.csv
063.csv
064.csv
066.csv
070.csv
071.csv
073.csv
082.csv
083.csv
088.csv
090.csv
091.csv
092.csv
094.csv
096.csv
102.csv
104.csv
108.csv
110.csv
113.csv
114.csv
117.csv
119.csv
127.csv
129.csv
131.csv
135.csv
137.csv
140.csv
143.csv
148.csv


In [16]:
df_vib = vib_df.reset_index()
df_vib = df_vib.drop(['index'], axis = 'columns')

In [17]:
df_vib.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.001162,0.001894,0.002627,0.001040,-0.000669,0.000429,0.000796,0.000307,0.001772,-0.000547,...,0.000551,0.002505,0.001406,-0.001402,-0.000181,0.000918,-0.001402,-0.002378,-0.000669,-0.001157
1,0.002066,0.002676,0.000235,0.000113,0.000845,0.002676,0.002188,-0.000742,-0.001352,0.000235,...,0.000235,-0.000253,-0.001230,-0.001596,-0.002328,-0.000619,-0.001474,-0.001962,-0.000009,0.001212
2,-0.000062,0.001281,0.003478,0.003234,0.000305,0.000549,0.000427,0.001037,0.000793,0.001281,...,0.001525,-0.000062,-0.001282,0.000182,0.000060,-0.000794,0.000182,0.000305,0.001403,-0.000428
3,-0.000765,-0.000765,-0.000643,-0.000887,-0.001376,-0.001620,-0.001742,-0.001376,-0.000765,-0.000765,...,-0.000155,-0.000155,-0.001376,-0.001742,-0.000521,-0.000033,-0.000643,-0.001009,-0.000643,0.000089
4,0.001327,0.004379,0.002304,-0.003800,-0.000870,0.001938,0.001938,0.001083,0.003036,0.001938,...,-0.001725,0.001327,0.000839,-0.002579,-0.000748,0.000839,-0.002335,-0.000748,0.001327,0.002182


In [18]:
print(df_vib.shape)

(56, 60)


In [19]:
df_vib.to_csv('vib_test.csv')

### current

In [30]:
curr_r_df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,...,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.120117,0.080078,0.120117
0,-2.872803,-2.912842,-2.672607,-2.192139,-1.831787,-1.311279,-0.990967,-0.550537,0.010010,0.650635,...,2.692627,2.692627,2.532471,2.212158,1.851807,1.411377,1.091064,0.650635,0.130127,-0.430420
0,-5.805664,-4.844727,-4.444336,-3.283203,-2.762695,-1.521484,-0.480469,0.800781,2.282227,3.603516,...,5.725586,4.804688,4.284180,3.323242,2.602539,1.321289,0.400391,-1.041016,-2.202148,-3.803711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,...,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156,-0.160156
0,-5.125000,4.564453,7.126953,15.775391,19.418945,25.344727,29.708984,33.792969,37.076172,38.037109,...,10.930664,19.058594,23.502930,28.547852,31.270508,35.074219,38.958008,42.441406,44.563477,45.244141
0,-0.120117,-0.120117,-0.120117,-0.160156,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117,...,-0.120117,-0.160156,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117,-0.120117
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.040039,0.000000,0.000000,0.000000,0.000000,0.000000,0.040039
0,-0.040039,-0.040039,-0.040039,-0.080078,-0.040039,-0.040039,-0.080078,-0.080078,-0.080078,-0.040039,...,-0.040039,-0.080078,-0.080078,-0.080078,-0.080078,-0.040039,-0.080078,-0.040039,-0.040039,-0.080078
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
for i, index in enumerate(Curr_index):
    print(file_list[index])
    if index == 141:
        print('error')
        pass
    else:
        one_signal = pd.read_csv(os.path.join(folder_name, file_list[index]))
        r_signal = pd.DataFrame(one_signal.iloc[:,1].to_numpy().reshape(1,-1))
        t_signal = pd.DataFrame(one_signal.iloc[:,2].to_numpy().reshape(1,-1))
        s_signal = pd.DataFrame(one_signal.iloc[:,3].to_numpy().reshape(1,-1))
        if i == 0:
            curr_r_df = pd.DataFrame(r_signal)
            curr_t_df = pd.DataFrame(t_signal)
            curr_s_df = pd.DataFrame(s_signal)
        else:
            curr_r_df = pd.concat([curr_r_df, r_signal])
            curr_t_df = pd.concat([curr_t_df, t_signal])
            curr_s_df = pd.concat([curr_s_df, s_signal])

004.csv
006.csv
013.csv
017.csv
022.csv
031.csv
034.csv
035.csv
037.csv
040.csv
042.csv
047.csv
050.csv
055.csv
060.csv
061.csv
065.csv
067.csv
068.csv
075.csv
076.csv
078.csv
080.csv
081.csv
087.csv
089.csv
093.csv
095.csv
097.csv
098.csv
101.csv
103.csv
105.csv
107.csv
111.csv
112.csv
115.csv
116.csv
118.csv
121.csv
124.csv
126.csv
136.csv
138.csv
139.csv
142.csv
error
145.csv
146.csv
147.csv
150.csv


In [35]:
df_curr_r = curr_r_df.reset_index()
df_curr_r = df_curr_r.drop(['index'], axis = 'columns')

In [36]:
df_curr_t = curr_t_df.reset_index()
df_curr_t = df_curr_t.drop(['index'], axis = 'columns')

In [37]:
df_curr_s = curr_s_df.reset_index()
df_curr_s = df_curr_s.drop(['index'], axis = 'columns')

In [39]:
print(df_curr_r.shape)
print(df_curr_t.shape)
print(df_curr_s.shape)

(49, 60)
(49, 60)
(49, 60)


In [41]:
df_curr_r.to_csv('curr_R_test.csv')
df_curr_t.to_csv('curr_T_test.csv')
df_curr_s.to_csv('curr_S_test.csv')

## 모델 적용 

In [ ]:
#normalization
def z_norm(data):
    std_ = data.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    norm_data = (data - data.mean(axis=1, keepdims=True)) / std_
    return norm_data

### kimm

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np

In [ ]:
kimm_model = keras.models.load_model('models/kimm_best_model.h5',
                                custom_objects=None, compile=True)

In [ ]:
kimm_sample = tf.convert_to_tensor(z_norm(df_kimm), dtype=tf.float64)

In [ ]:
y_pred = kimm_model.predict(kimm_sample)
y_kimm_pred = np.argmax(y_pred,axis=1)
print(y_kimm_pred)

### vibration

In [ ]:
vib_model = keras.models.load_model('models/vib_best_model.h5',
                                custom_objects=None, compile=True)

In [ ]:
vib_sample = tf.convert_to_tensor(z_norm(df_vib), dtype=tf.float64)

In [ ]:
y_pred = vib_model.predict(vib_sample)
y_vib_pred = np.argmax(y_pred,axis=1)
print(y_vib_pred)

### current

In [ ]:
curr_model = keras.models.load_model('models/kimm_best_model.h5',
                                custom_objects=None, compile=True)

In [ ]:
cur_R_sample = tf.convert_to_tensor(z_norm(df_curr_r), dtype=tf.float64)
cur_T_sample = tf.convert_to_tensor(z_norm(df_curr_t), dtype=tf.float64)
cur_S_sample = tf.convert_to_tensor(z_norm(df_curr_s), dtype=tf.float64)

In [ ]:
y_pred = curr_model.predict(cur_R_sample,cur_T_sample, cur_S_sample)
y_curr_pred = np.argmax(y_pred,axis=1)
print(y_curr_pred)